to know how many cases are in this file

In [71]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


get the title list

In [38]:
# import fitz
# import re

# def extract_title(pdf_path, patterns):
# # def extract_title(pdf_path):
#     doc = fitz.open(pdf_path)
#     extracted_text_list = []

#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
#         blocks = page.get_text("dict")["blocks"]

#         for b in blocks:
#             if "lines" in b:
#                 for line in b["lines"]:
#                     for span in line["spans"]:
#                         if 'font' in span and span['font'] == 'Helvetica-Oblique':
#                             # result_string = re.sub(r',\s*\d+\s*F\.\d+\s*\d*', '', span["text"])
#                             # extracted_text_list.append(result_string.strip())
#                             result_string = span["text"].strip()
#                             # Try matching each pattern
#                             for pattern in patterns:
#                                 result_string = re.sub(pattern, '', result_string)
#                             extracted_text_list.append(result_string)    

#         if len(extracted_text_list) >= extract_target_number(pdf_path):
#             break

#     doc.close()
#     return extracted_text_list

In [46]:
# import fitz
# import re

# def extract_title(pdf_path):
#     doc = fitz.open(pdf_path)
#     extracted_text_list = []

#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
#         blocks = page.get_text("dict")["blocks"]

#         for b in blocks:
#             if "lines" in b:
#                 for line in b["lines"]:
#                     for span in line["spans"]:
#                         if 'font' in span and span['font'] == 'Helvetica-Oblique':
#                             pattern = r'(.*),(?=\s*\d+\s*[A-Za-z. ]+\s*\d+$)'
#                             match = re.match(pattern, span["text"])
#                             if match:
#                                 extracted_text = match.group(1).strip()
#                                 print(extracted_text)
#                                 extracted_text_list.append(extracted_text)    

#         if len(extracted_text_list) >= extract_target_number(pdf_path):
#             break

#     doc.close()
#     return extracted_text_list

In [72]:
import fitz
import re

def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            result_string = re.sub(r',[^,]*$', '', span["text"].strip())
                            extracted_text_list.append(result_string)

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list


In [78]:

pdf_file_path = "D:/antitrust/verdict/Files (35).PDF"
result_list = extract_title(pdf_file_path)

In [79]:
len(result_list)

35

In [75]:
import csv

get other information and write them into csv

In [76]:
import fitz

def getAndWrite(pdf_path, case_list, output_csv):

    doc = fitz.open(pdf_path)
    matching_texts = []
    # date = ''
    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = None
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                for case_name in case_list:
                                    if span["text"].strip() == case_name.strip():
                                        title = span["text"].strip()
                                        # print(title)
                                        # Set the flag to consider 'number': 2 and 'number': 3 text
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
                                # print(court)
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                # date = span["text"].strip()
                                # date += span["text"].strip()
                                date = date.strip() + span["text"].strip() if date else span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                # Stop considering 'number': 2, 'number': 3, and 'number': 4 text
                                consider_number_2 = False
                                print({'Title': title, 'Court': court, 'Date': date, 'No': no, 'page':page_num+1 })
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'Title': title, 'Court': court, 'Date': date, 'No': no, 'page':page_num+1 })
    
    doc.close()
    
    # Save matching_texts to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Court', 'Date', 'No', 'page']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)



In [80]:
result_texts = getAndWrite(pdf_file_path, result_list, 'Files (35).csv')


{'Title': 'Hartford Fire Ins. Co. v. Cal.', 'Court': 'Supreme Court of the United States', 'Date': 'February 23, 1993, Argued ; June 28, 1993, Decided', 'No': 'No. 91-1111', 'page': 8}
{'Title': 'United States v. Lopez', 'Court': 'Supreme Court of the United States', 'Date': 'November 8, 1994, Argued ; April 26, 1995, Decided', 'No': 'No. 93-1260', 'page': 44}
{'Title': 'Brown v. Pro Football', 'Court': 'Supreme Court of the United States', 'Date': 'March 27, 1996, Argued ; June 20, 1996, Decided', 'No': 'No. 95-388', 'page': 97}
{'Title': 'Klehr v. A.O. Smith Corp.', 'Court': 'Supreme Court of the United States', 'Date': 'April 21, 1997, Argued ; June 19, 1997, Decided', 'No': 'No. 96-663', 'page': 121}
{'Title': 'State Oil Co. v. Khan', 'Court': 'Supreme Court of the United States', 'Date': 'October 7, 1997, Argued ; November 4, 1997, Decided', 'No': 'No. 96-871', 'page': 137}
{'Title': 'Nynex Corp. v. Discon', 'Court': 'Supreme Court of the United States', 'Date': 'October 5, 1998, 